# Amazon Barkeley Products Data

In [31]:
!pip install langchain==0.0.123
!pip install openai==0.27.2
!pip install redis==4.5.3
!pip install numpy
!pip install pandas
!pip install gdown

In [32]:
import pandas as pd
df = pd.read_csv('ecom_product_data.csv')       #144 MB database
df.head(10)

,item_id,marketplace,country,main_image_id,domain_name,bullet_point,item_keywords,material,brand,color,item_name,model_name,model_number,product_type
0,B07T6RZ2CM,Amazon,IN,71dZhpsferL,amazon.in,3D Printed Hard Back Case Mobile Cover for Len...,mobile cover back cover mobile case phone case...,NaN,Amazon Brand - Solimo,Others,Amazon Brand - Solimo Designer Couples Sitting...,Lenovo K4 Note,gz8115-SL40423,CELLULAR_PHONE_CASE
1,B07T2JY31Y,Amazon,IN,71vX7qIEAIL,amazon.in,3D Printed Hard Back Case Mobile Cover for Son...,mobile cover back cover mobile case phone case...,Wood,Amazon Brand - Solimo,others,Amazon Brand - Solimo Designer Leaf on Wood 3D...,Sony Xperia Z1 L39H,gz8056-SL40528,CELLULAR_PHONE_CASE
2,B0849YGSCZ,Amazon,AE,A1EZF-2mB5L,amazon.ae,NaN,small de fur rooms navidad woven girls shag pa...,NaN,Stone & Beam,NaN,Stone & Beam Contemporary Doily Wool Farmhouse...,NaN,I59I8044IVYGRYC00-Parent,HOME_FURNITURE_AND_DECOR
3,B081K6TCML,Amazon,IN,81o9EyZ-fAL,amazon.in,Solimo Plastic Multipurpose Modular Drawer; sm...,drawer modular drawer 3 rack modular drawer ki...,Plastic,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Plastic Multipurpose Mod...,NaN,sol_cujo_13,HOME
4,B0854774X5,Amazon,IN,81xaJCVnl3L,amazon.in,"Snug fit for Nokia 8.1, with perfect cut-outs ...",Back Cover Designer Case Designer Take It Easy...,Silicon,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Take It Easy UV...,Nokia 8.1,UV10714-SL40617,CELLULAR_PHONE_CASE
5,B07NJPBZ19,Amazon,US,71m7q8wxdUL,amazon.com,One 21-fluid ounce bottle of body wash with mo...,Solimo Cool Mist Body Wash body wash body wash...,NaN,Solimo,NaN,"Amazon Brand - Solimo Body Wash, Cool Mist Sce...",NaN,NaN,SKIN_CLEANING_AGENT
6,B013R4544C,Amazon,CA,81agKrncxwL,amazon.ca,An Amazon brand,cold weather gloves cold weather gear cold wea...,NaN,Thirty Five Kent,Blue,"Thirty Five Kent Men's Cashmere Zig Zag Scarf,...",NaN,NaN,ACCESSORY
7,B086B477BH,Amazon,ES,51ZjeCCBkhL,amazon.es,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5400606985223,WINE
8,B07THQWPJZ,Amazon,IN,81AyD23O40L,amazon.in,3D Printed Hard Back Case Mobile Cover for Son...,mobile cover back cover mobile case phone case...,NaN,Amazon Brand - Solimo,Others,Amazon Brand - Solimo Designer Rangolis 3D Pri...,Sony Xperia L1,gz8467-SL40707,CELLULAR_PHONE_CASE
9,B07T9TMRXJ,Amazon,IN,813KjyvHJfL,amazon.in,3D Printed Hard Back Case Mobile Cover for HTC...,mobile cover back cover mobile case phone case...,NaN,Amazon Brand - Solimo,Others,Amazon Brand - Solimo Designer No Hate On Wood...,HTC U Ultra,gz8405-SL40527,CELLULAR_PHONE_CASE


In [33]:
# data = pd.read_csv('product_data.csv')
# data.head()

In [34]:
df.shape

(147702, 14)

In [35]:
print("1. Unique marketplaces - ", df.marketplace.unique())      #first letter capitalise
print("2. Unique Countries" , df.country.unique())
print("3. Unique Domain names", df.domain_name.unique())
print("4. Unique Materials", df.material.unique())
print("5. Unique Brands", len(df.brand.unique()))
print("6. Unique Product_Types", len(df.product_type.unique()))       #Upper case countries

1. Unique marketplaces -  ['Amazon' 'PrimeNow' 'AmazonFresh' 'AmazonGo' 'WholeFoods'
 'AmazonDistribution' 'Woot']
2. Unique Countries ['IN' 'AE' 'US' 'CA' 'ES' 'MX' 'DE' 'UK' 'NL' 'FR' 'IT' 'JP' 'GB' 'SA'
 'SG' 'SE' 'AU' 'TR' 'PL' 'BR']
3. Unique Domain names ['amazon.in' 'amazon.ae' 'amazon.com' 'amazon.ca' 'amazon.es'
 'amazon.com.mx' 'amazon.de' 'primenow.amazon.co.uk' 'amazon.nl'
 'amazon.fr' 'amazon.it' 'amazon.co.jp' 'amazon.co.uk' 'fresh.amazon.com'
 'amazon.sa' 'amazon.sg' 'amazon.com/go' 'primenow.amazon.com'
 'primenow.amazon.de' 'wholefoodsmarket.com' 'amazon.se'
 'primenow.amazon.it' 'amazon.com.au' 'amazon.com.tr' 'primenow.amazon.fr'
 'primenow.amazon.ca' 'primenow.amazon.es' 'primenow.amazon.co.jp'
 'amazon.pl' 'amazon.com.br' 'amazondistribution.in' 'woot.com']
4. Unique Materials [nan 'Wood' 'Plastic' ... 'Metal / Metal' 'Vinyl Vinyl'
 'Alloy Steel Plastic']
5. Unique Brands 408
6. Unique Product_Types 576


In [36]:
df.bullet_point = df.bullet_point.fillna("")
df.item_keywords = df.item_keywords.fillna("")
df.item_name = df.item_name.fillna("")

In [37]:
def max_len(i,res):
    res = max(res,len(i))
    return res

In [38]:
res=0
max_res=0
for i in df.bullet_point: 
    max_res = max(max_res,max_len(i,res))

print("Maximum Length of Bullet points", max_res)

Maximum Length of Bullet points 3774


In [39]:
res=0
max_res=0
for i in df.item_keywords: 
    max_res = max(max_res,max_len(i,res))

print("Maximum Length of Item_keywords", max_res)

Maximum Length of Item_keywords 118647


In [40]:
res=0
max_res=0
for i in df.item_name: 
    max_res = max(max_res,max_len(i,res))

print("Maximum Length of Item_name", max_res)

Maximum Length of Item_name 409


## Preprocessing

In [41]:
MAX_TEXT_LENGTH=10000

def auto_truncate(val):
    
    return val[:MAX_TEXT_LENGTH]


all_prods_df = pd.read_csv("ecom_product_data.csv", converters={
 
    'item_keywords': auto_truncate,
 
})

In [42]:
all_prods_df

,item_id,marketplace,country,main_image_id,domain_name,bullet_point,item_keywords,material,brand,color,item_name,model_name,model_number,product_type
0,B07T6RZ2CM,Amazon,IN,71dZhpsferL,amazon.in,3D Printed Hard Back Case Mobile Cover for Len...,mobile cover back cover mobile case phone case...,NaN,Amazon Brand - Solimo,Others,Amazon Brand - Solimo Designer Couples Sitting...,Lenovo K4 Note,gz8115-SL40423,CELLULAR_PHONE_CASE
1,B07T2JY31Y,Amazon,IN,71vX7qIEAIL,amazon.in,3D Printed Hard Back Case Mobile Cover for Son...,mobile cover back cover mobile case phone case...,Wood,Amazon Brand - Solimo,others,Amazon Brand - Solimo Designer Leaf on Wood 3D...,Sony Xperia Z1 L39H,gz8056-SL40528,CELLULAR_PHONE_CASE
2,B0849YGSCZ,Amazon,AE,A1EZF-2mB5L,amazon.ae,NaN,small de fur rooms navidad woven girls shag pa...,NaN,Stone & Beam,NaN,Stone & Beam Contemporary Doily Wool Farmhouse...,NaN,I59I8044IVYGRYC00-Parent,HOME_FURNITURE_AND_DECOR
3,B081K6TCML,Amazon,IN,81o9EyZ-fAL,amazon.in,Solimo Plastic Multipurpose Modular Drawer; sm...,drawer modular drawer 3 rack modular drawer ki...,Plastic,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Plastic Multipurpose Mod...,NaN,sol_cujo_13,HOME
4,B0854774X5,Amazon,IN,81xaJCVnl3L,amazon.in,"Snug fit for Nokia 8.1, with perfect cut-outs ...",Back Cover Designer Case Designer Take It Easy...,Silicon,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Take It Easy UV...,Nokia 8.1,UV10714-SL40617,CELLULAR_PHONE_CASE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147697,B07TG4C758,Amazon,IN,61TY726nPrL,amazon.in,3D Printed Hard Back Case Mobile Cover for Viv...,mobile cover back cover mobile case phone case...,NaN,Amazon Brand - Solimo,Others,Amazon Brand - Solimo Designer Round Candy 3D ...,Vivo Y55L,gz8291-SL40643,CELLULAR_PHONE_CASE
147698,B07TF1FNHR,Amazon,IN,61kDV33nSFL,amazon.in,3D Printed Hard Back Case Mobile Cover for Sam...,mobile cover back cover mobile case phone case...,NaN,Amazon Brand - Solimo,Others,Amazon Brand - Solimo Designer Desert Photogra...,Samsung Galaxy J6 Plus,gz8588-SL40393,CELLULAR_PHONE_CASE
147699,B0853XL62C,Amazon,IN,81Bp5uyxi4L,amazon.in,"Snug fit for Itel A22 Pro, with perfect cut-ou...",Back Cover Designer Case Designer Wooden Art M...,Silicone,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Wooden Art UV P...,Itel A22 Pro,UV10597-SL40593,CELLULAR_PHONE_CASE
147700,B081HNLHQ5,Amazon,IN,61lQ7vaD0+L,amazon.in,"Snug fit for Mobile, with perfect cut-outs for...",mobile cover back cover mobile case phone case...,Plastic,Amazon Brand - Solimo,multi-colored,Amazon Brand - Solimo Designer Girl Design 3D ...,Samsung Galaxy A50s,SOLSA50S3DM15,CELLULAR_PHONE_CASE


In [43]:
all_prods_df['item_keywords'].replace('', None, inplace=True)
all_prods_df.dropna(subset=['item_keywords'], inplace=True)

all_prods_df.reset_index(drop=True, inplace=True)

In [44]:
all_prods_df.head()

,item_id,marketplace,country,main_image_id,domain_name,bullet_point,item_keywords,material,brand,color,item_name,model_name,model_number,product_type
0,B07T6RZ2CM,Amazon,IN,71dZhpsferL,amazon.in,3D Printed Hard Back Case Mobile Cover for Len...,mobile cover back cover mobile case phone case...,NaN,Amazon Brand - Solimo,Others,Amazon Brand - Solimo Designer Couples Sitting...,Lenovo K4 Note,gz8115-SL40423,CELLULAR_PHONE_CASE
1,B07T2JY31Y,Amazon,IN,71vX7qIEAIL,amazon.in,3D Printed Hard Back Case Mobile Cover for Son...,mobile cover back cover mobile case phone case...,Wood,Amazon Brand - Solimo,others,Amazon Brand - Solimo Designer Leaf on Wood 3D...,Sony Xperia Z1 L39H,gz8056-SL40528,CELLULAR_PHONE_CASE
2,B0849YGSCZ,Amazon,AE,A1EZF-2mB5L,amazon.ae,NaN,small de fur rooms navidad woven girls shag pa...,NaN,Stone & Beam,NaN,Stone & Beam Contemporary Doily Wool Farmhouse...,NaN,I59I8044IVYGRYC00-Parent,HOME_FURNITURE_AND_DECOR
3,B081K6TCML,Amazon,IN,81o9EyZ-fAL,amazon.in,Solimo Plastic Multipurpose Modular Drawer; sm...,drawer modular drawer 3 rack modular drawer ki...,Plastic,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Plastic Multipurpose Mod...,NaN,sol_cujo_13,HOME
4,B0854774X5,Amazon,IN,81xaJCVnl3L,amazon.in,"Snug fit for Nokia 8.1, with perfect cut-outs ...",Back Cover Designer Case Designer Take It Easy...,Silicon,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Take It Easy UV...,Nokia 8.1,UV10714-SL40617,CELLULAR_PHONE_CASE


In [55]:
all_prods_df = all_prods_df.sample(n=2500)

In [57]:
all_prods_df.to_csv('ecom_product_data.csv',index=False)

In [46]:
NUMBER_PRODUCTS = 2500  

product_metadata = ( 
    all_prods_df
     .head(NUMBER_PRODUCTS)
     .to_dict(orient='index')
)

product_metadata[2]                    #5.5MB size

{'item_id': 'B0849YGSCZ',
 'marketplace': 'Amazon',
 'country': 'AE',
 'main_image_id': 'A1EZF-2mB5L',
 'domain_name': 'amazon.ae',
 'bullet_point': nan,
 'item_keywords': 'small de fur rooms navidad woven girls shag pastel white furniture blush plaid runner pale sheepskin safavieh 10 area large dorm nuloom 4 muddle pink 8x10 yellow for trellis ft colors fauxfur decor and ivory indoors clearance rugs colored 9x12 kitchen floor furry fuzzy mat teen 2x3 room round',
 'material': nan,
 'brand': 'Stone & Beam',
 'color': nan,
 'item_name': 'Stone & Beam Contemporary Doily Wool Farmhouse Rug',
 'model_name': nan,
 'model_number': 'I59I8044IVYGRYC00-Parent',
 'product_type': 'HOME_FURNITURE_AND_DECOR'}

In [47]:
import json

In [48]:
json_res = json.dumps(product_metadata)

In [49]:
# with open("data/product_metadata.json","w") as f:
#     json.dump(json_res,f)

In [50]:
import os
 
from langchain.embeddings import OpenAIEmbeddings
 
os.environ['OPENAI_API_KEY'] = ""
 
texts = [
    v['item_name'] for k, v in product_metadata.items()
]

In [51]:
metadatas = list(product_metadata.values())

In [22]:
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import (
    ConversationalRetrievalChain,
    LLMChain
)
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts.prompt import PromptTemplate

In [23]:
template = """Given the following chat history and a follow up question, rephrase the follow up input question to be a standalone question.
Or end the conversation if it seems like it's done.
Chat History:\"""
{chat_history}
\"""
Follow Up Input: \"""
{question}
\"""
Standalone question:"""
 
condense_question_prompt = PromptTemplate.from_template(template)
 
template = """You are a friendly, conversational retail shopping assistant. Use the following context including product names, descriptions, and keywords to show the shopper whats available, help find what they want, and answer any questions.
 
It's ok if you don't know the answer.
Context:\"""
 
{context}
\"""
Question:\"
\"""
 
Helpful Answer:"""
 
qa_prompt= PromptTemplate.from_template(template)

In [24]:
# define two LLM models from OpenAI
llm = OpenAI(temperature=0)
 
streaming_llm = OpenAI(
    streaming=True,
    callback_manager=CallbackManager([
        StreamingStdOutCallbackHandler()
    ]),
    verbose=True,
    max_tokens=150,
    temperature=0.2
)
 
# use the LLM Chain to create a question creation chain
question_generator = LLMChain(
    llm=llm,
    prompt=condense_question_prompt
)
 
# use the streaming LLM to create a question answering chain
doc_chain = load_qa_chain(
    llm=streaming_llm,
    chain_type="stuff",
    prompt=qa_prompt
)

In [25]:
embeddings = OpenAIEmbeddings()

## Redis

In [62]:
from langchain.vectorstores.redis import Redis as RedisVectorStore

# name of the Redis search index to create
index_name = "products"

# assumes you have a redis stack server running on local host
redis_url = "redis://redis-service.samiksha-kohle.cluster.local:6379"

In [63]:
vectorstore = RedisVectorStore.from_texts(
    texts=texts,
    metadatas=metadatas,
    embedding=embeddings,
    index_name=index_name,
    redis_url=redis_url
)

In [64]:
chatbot = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(),
    combine_docs_chain=doc_chain,
    question_generator=question_generator
)

In [65]:
chat_history = []
question = input("Hi! What are you looking for today?")

while True:
    
    result = chatbot(
        {"question": question, "chat_history": chat_history}
    )
    
    print("\n")
    chat_history.append((result["question"], result["answer"]))
    question = input()

Hi! What are you looking for today? gold plated earings


 Hi there! We have a great selection of earrings available. We have the Yellow Gold-Plated Sterling Silver Swarovski Zirconia Fancy Green Stud Earrings, which feature a beautiful green Swarovski Zirconia stone set in a yellow gold-plated sterling silver setting. We also have the Platinum or Gold-Plated Sterling Silver Princess-Cut Swarovski Zirconia Hoop Earrings, which feature a princess-cut Swarovski Zirconia stone set in a platinum or gold-plated sterling silver setting. Let me know if you have any questions about these earrings or if you'd like to see more options!



 price of gold plated earings


 Hi there! We have a great selection of earrings for you to choose from. We have Yellow Gold-Plated Sterling Silver Swarovski Zirconia Fancy Green Stud Earrings, and Platinum or Gold-Plated Sterling Silver Princess-Cut Swarovski Zirconia Hoop Earrings. Both of these earrings feature beautiful Swarovski Zirconia stones in a variety of colors. Let me know if you have any questions or if you'd like to see more options!



 My preference is the yellow gold plated sterling silver


...

 Comfortable walking shoes




Hi there! I see you're looking for walking shoes and slippers. We have the Symactive Men's Navy Walking Shoes-9 UK (43 EU) (10 US) (SYM-SS-028C) and the 206 Collective Women's Roosevelt Shearling Slide Slipper Shoe, Chestnut, 7.5 B US. Both are from our Amazon Brand and are great quality. Let me know if you have any questions about either of these products.



 A durable iPhone case


 Hi there! It looks like you're looking for a mobile cover for your Apple iPhone 8 Plus. We have two options available from Amazon Brand - Solimo. The first is a Designer Sky Blue and Orange Canvas 3D Printed Hard Back Case Mobile Cover with Logo Cut. The second is a Designer Universe Printed Hard Back Case Mobile Cover with a D1258 model number. Let me know if you have any questions about either of these options.



KeyboardInterrupt: Interrupted by user

In [68]:
import json
from langchain.schema import BaseRetriever
from langchain.vectorstores import VectorStore
from langchain.schema import Document
from pydantic import BaseModel

class RedisProductRetriever(BaseRetriever, BaseModel):
    vectorstore: VectorStore
 
    class Config:
        arbitrary_types_allowed = True
 
    def combine_metadata(self, doc) -> str:
        metadata = doc.metadata
        return (
           "Item Name: " + metadata["item_name"] + ". " +
           "Item Description: " + metadata["bullet_point"] + ". " +
           "Item Keywords: " + metadata["item_keywords"] + "."
        )
 
    def get_relevant_documents(self, query):
        docs = []
        for doc in self.vectorstore.similarity_search(query):
            content = self.combine_metadata(doc)
            docs.append(Document(
                page_content=content,
                metadata=doc.metadata
            ))
 
        return docs

In [69]:
redis_product_retriever = RedisProductRetriever(vectorstore=vectorstore)

chatbot = ConversationalRetrievalChain(
    retriever=pinecone_product_retriever,
    combine_docs_chain=doc_chain,
    question_generator=question_generator
)

TypeError: Can't instantiate abstract class RedisProductRetriever with abstract method aget_relevant_documents

In [ ]:
# def e_com_chatbot(condense_question_prompt,qa_prompt,):
#     #embeddings
#     embeddings = OpenAIEmbeddings()
    
#     text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
#     documents = text_splitter.split_documents(document)
#     vectorstore = FAISS.from_documents(documents, embeddings)
   
#     #gpt
#     llm = OpenAI(temperature=0)

#     streaming_llm = OpenAI(
#         streaming=True,
#         callback_manager=CallbackManager([
#             StreamingStdOutCallbackHandler()
#         ]),
#         verbose=True,
#         max_tokens=150,
#         temperature=0.2
#     )
    
#     question_generator = LLMChain(
#         llm=llm,
#         prompt=condense_question_prompt
#     )
    
#     doc_chain = load_qa_chain(
#         llm=streaming_llm,
#         chain_type="stuff",
#         prompt=qa_prompt
#     )
    
#     chatbot = ConversationalRetrievalChain(
#         retriever=vectorstore.as_retriever(),
#         combine_docs_chain=doc_chain,
#         question_generator=question_generator
#     )
    
#     return chatbot

## Pinecone

In [54]:
from langchain.vectorstores import Pinecone

In [46]:
# class Document:
#     def __init__(self, page_content, metadata):
#         self.page_content = page_content
#         self.metadata = metadata
        
#     def __str__(self):
#             return f"Document(page_content='{self.page_content}', metadata={self.metadata})"

In [82]:
from langchain.schema import Document
import pinecone

In [83]:
pinecone.init(
            api_key="",  # find at app.pinecone.io
            environment="asia-southeast1-gcp-free",  # next to api key in console
        )

In [84]:
def create_index_if_not_exists(index_name, dimension, metric):
    if index_name not in pinecone.list_indexes():
        pinecone.create_index(index_name, dimension=dimension, metric=metric)
        print(f"Index '{index_name}' created successfully.")
    else:
        print(f"Index '{index_name}' already exists.")

In [ ]:
!pip install pinecone-client

In [88]:
#Method1

### from langchain.schema import Document
import time
doc_lst = []

index_name = "ecom-chatbot"

create_index_if_not_exists(index_name,)

st_t = time.time()
for files in range(len(texts[0])):
    document = Document(page_content=texts[files], metadata=metadatas[files])
    doc_lst.append(document) 
    vectorstore = Pinecone.from_documents(doc_lst, embeddings, index_name=index_name)
    doc_lst.pop()
    
print("total_time", time.time()-st_t)

TypeError: create_index_if_not_exists() missing 2 required positional arguments: 'dimension' and 'metric'

In [ ]:
#Method2: upserting with index



In [51]:
import json
from langchain.schema import BaseRetriever
from langchain.vectorstores import VectorStore
from langchain.schema import Document
from pydantic import BaseModel

class PineconeProductRetriever(BaseRetriever, BaseModel):
    vectorstore: VectorStore
 
    class Config:
        arbitrary_types_allowed = True
 
    def combine_metadata(self, doc) -> str:
        metadata = doc.metadata
        return (
           "Item Name: " + metadata["item_name"] + ". " +
           "Item Description: " + metadata["bullet_point"] + ". " +
           "Item Keywords: " + metadata["item_keywords"] + "."
        )
 
    def get_relevant_documents(self, query):
        docs = []
        for doc in self.vectorstore.similarity_search(query):
            content = self.combine_metadata(doc)
            docs.append(Document(
                page_content=content,
                metadata=doc.metadata
            ))
 
        return docs

In [26]:
llm = OpenAI(temperature=0)

streaming_llm = OpenAI(
    streaming=True,
    callback_manager=CallbackManager([
        StreamingStdOutCallbackHandler()
    ]),
    verbose=True,
    max_tokens=150,
    temperature=0.2
)

question_generator = LLMChain(
    llm=llm,
    prompt=condense_question_prompt
)

doc_chain = load_qa_chain(
    llm=streaming_llm,
    chain_type="stuff",
    prompt=qa_prompt
)

In [66]:
pinecone_product_retriever = PineconeProductRetriever(vectorstore=vectorstore)

chatbot = ConversationalRetrievalChain(
    retriever=pinecone_product_retriever,
    combine_docs_chain=doc_chain,
    question_generator=question_generator
)

NameError: name 'vectorstore' is not defined

## FAISS 

In [46]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 66.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [29]:
from langchain.schema import Document

In [41]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
import time

doc_lst = []

st_t = time.time()

for files in range(len(texts[:1000])):
    document = Document(page_content=texts[files], metadata=metadatas[files])
    doc_lst.append(document)
    print(doc_lst)
    str_time = time.time()
    vectorstore = FAISS.from_documents(doc_lst, embeddings)
    print("time_req",time.time()-str_time)
    doc_lst.pop()

print("Total_time_required",time.time()-st_t)

[Document(page_content='Amazon Brand - Solimo Designer Couples Sitting at Dark 3D Printed Hard Back Case Mobile Cover for Lenovo K4 Note', lookup_str='', metadata={'item_id': 'B07T6RZ2CM', 'marketplace': 'Amazon', 'country': 'IN', 'main_image_id': '71dZhpsferL', 'domain_name': 'amazon.in', 'bullet_point': '3D Printed Hard Back Case Mobile Cover for Lenovo K4 Note Easy to put & take off with perfect cutouts for volume buttons, audio & charging ports. Stylish design and appearance, express your unique personality. Extreme precision design allows easy access to all buttons and ports while featuring raised bezel to life screen and camera off flat surface. Slim Hard Back Cover No Warranty None', 'item_keywords': 'mobile cover back cover mobile case phone case mobile panel phone panel Lenovo mobile case Lenovo phone cover Lenovo back case hard case 3D printed mobile cover mobile cover back cover mobile case phone case mobile panel phone panel Lenovo mobile case Lenovo phone cover Lenovo back

In [42]:
chatbot = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(),
    combine_docs_chain=doc_chain,
    question_generator=question_generator
)

### Prompt Template

In [ ]:
print("sdmfnjgb")

In [43]:
chat_history = []
question = input("Hi! What are you looking for today?")

while True:
    
    result = chatbot(
        {"question": question, "chat_history": chat_history}
    )
    
    print("\n")
    chat_history.append((result["question"], result["answer"]))
    question = input()

Hi! What are you looking for today? gold plated ear rings


 Hi there! I see you're looking for a mobile cover for your Lava Z70. We have the Amazon Brand - Solimo Designer Love UV Printed Soft Back Case Mobile Cover for Lava Z70. It's made of a soft material and features a UV printed design. It's designed to provide protection for your phone and keep it looking great. Let me know if you have any other questions about this product!



 its pricing




Hi there! I'm here to help you find the perfect mobile cover for your Lava Z70. We have the Amazon Brand - Solimo Designer Love UV Printed Soft Back Case Mobile Cover for Lava Z70. This mobile cover is designed to provide protection from scratches and minor impacts. It also features a unique UV printed design that adds a stylish look to your phone. Let me know if you have any questions about this product!



 price


 Hi there! It looks like you're interested in the Amazon Brand - Solimo Designer Love UV Printed Soft Back Case Mobile Cover for Lava Z70. This case is designed to provide protection for your Lava Z70 while also adding a stylish touch. It features a UV printed design and a soft back case for extra protection. If you have any questions about this product, please let me know.



 Lava Z70 price


 Hi there! It looks like you're interested in the Amazon Brand - Solimo Designer Love UV Printed Soft Back Case Mobile Cover for Lava Z70. This case is designed to provide your phone with protection from scratches and other damage. It also features a unique UV printed design that adds a stylish touch to your device. If you have any questions about this product, please let me know.



KeyboardInterrupt: Interrupted by user

# Using Llamindex